# Neural network for addition of 2 numbers
### Mehdi ATTAOUI

In [19]:
import numpy as np
import matplotlib.pyplot as plt

In [20]:
import random

## 1 . Prepare training and test data (typically loaded from file)

- Here we generate it

In [21]:
rangeData = 20                             # Numbers from [-rangeData,+rangeData]
lenData = 1000                             # How many pairs of numbers do we generate
testProportion = 0.3                       # 30% testing, 70% training
testEnd = round(lenData * testProportion)  # How many pairs of numbers are used for testing

- Generate 1000 pairs of numbers as 1000 seperate inputs for our network

In [22]:
dataIn = np.random.randint(-rangeData, rangeData+1, size=(lenData, 2))

- Generate the corresponding 1000 output values. These will be the sum of the two inputs.
- We do not tell the network that it is the sum. The network shall learn this by itself.

In [23]:
dataOut = dataIn[:,0] + dataIn[:,1]

- Adding a '1' element to each input pair (related to bias - more on this later)

In [24]:
dataIn = np.concatenate([np.ones([lenData,1]), dataIn], axis=1)	

- The final data sets and 1 example each

In [25]:
testingIn   = dataIn[0:testEnd]
testingOut  = dataOut[0:testEnd]
trainingIn  = dataIn[testEnd:]
trainingOut = dataOut[testEnd:]


In [26]:
print( trainingIn[:] )

[[  1.  -5. -13.]
 [  1.  17.  -9.]
 [  1.   3.   7.]
 ...
 [  1.  -2.  15.]
 [  1. -11.  12.]
 [  1. -16.  14.]]


## 2. Setting up neural network

![Addition_network.png](Addition_network.png)

Input layer length: 3 (1 bias + 2 numbers)

Output layer length: 1 (result)

### 2.1 Initialize weights: Numbers in the range from -2 to 2

- We need a starting point for our weights. Let's select them randomly.

In [27]:
np.random.seed(42)
weights = np.random.uniform(-2, 2, size=(3,))
weights = weights.astype(float)  

print(weights)

[-0.50183952  1.80285723  0.92797577]


### 2.2 Activation function

- Typically a monotonuous function that rescales a value to the range [0,1]
- Here it is not necessary (Comes in the other examples)

### 2.3 Calculate output of our neural network

The value of a neuron is given as the dot product of the two vectors: 
- weights 
- value of the neurons in the previous layer (including bias: value 1)

$ y = w_0 + w_1x_1 + w_2x_2 $

In [28]:
y = weights[0]*trainingIn[:,0] + weights[1] * trainingIn[:,1] + weights[2] * trainingIn[:,2]

print("weights:",weights)
print("\n")
print(trainingIn)
print("\n")
print("predicted output: \n \n",y)



weights: [-0.50183952  1.80285723  0.92797577]


[[  1.  -5. -13.]
 [  1.  17.  -9.]
 [  1.   3.   7.]
 ...
 [  1.  -2.  15.]
 [  1. -11.  12.]
 [  1. -16.  14.]]


predicted output: 
 
 [-2.15798106e+01  2.17949514e+01  1.14025625e+01 -1.74431531e+01
  3.11808977e+01 -1.74431531e+01  2.69657733e+01  4.94749415e+01
  6.60340076e+00 -1.10004170e+01 -9.72850289e+00  1.95419672e+01
 -1.93303659e+00 -3.03286252e+01 -2.56356521e+01 -4.49638603e+01
  8.76263230e-01  2.57293184e+00  4.29732224e+00 -1.85304118e+01
  1.22243497e+01  6.86887230e+00 -3.78898812e+00  4.14948198e+01
  6.17864629e+00 -1.44214760e+01  8.40625799e+00 -2.00677975e+01
 -1.35719671e+01 -3.10973181e+01  4.31914884e+01 -3.05410024e+01
 -7.81945704e+00 -6.99766990e+00 -4.06956416e+01 -3.07002854e+01
 -1.89274445e+01  3.13932749e+01  2.13979187e+01 -7.23541965e+00
  2.05761316e+01 -4.13050515e+01 -5.14065963e+01  2.40756574e+01
 -5.85731689e+00  3.52862940e+00 -6.99766990e+00 -4.40358845e+01
  4.77251786e+01  3.87108925e+01 

- At the end of this notebook (after training our network) we will have the weights

$ w_0 = 0, w_1 = 1, w_2 = 1 $

because then our output will be 

$ y = x_1 + x_2 $

### 2.4 Functions: Calculate accuracy and individual error

### - Accuracy: 
What is the rate at which the output is predicted correctly (only correct and wrong matter)?

In [29]:
correct_predictions = np.sum(y == trainingOut)

accuracy = correct_predictions / len(trainingOut)

print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 0.00%


- So far, output is random 

### - Error (better for learning): 
- For a pair of numbers we calculate:

$ \Delta = (y-Y)^2 $

$ y $: Predicted result by the neural network

$ Y $: Correct result (what we have calculated in the beginning)

- Here we only have a single output neuron but in general 

$ \Delta = (\vec{y}-\vec{Y})^2=\sum_j (y_j-Y_j)^2 $

In [30]:
errors = y - trainingOut

errors_squared = (y - trainingOut) ** 2

total_error = np.sum(errors_squared)

print(total_error)


63438.90312454851


In [31]:
def mse_loss(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

### 2.5 Function: Calculate gradient (d Error / d weight)

- All derivatives with respect to the individual weights (use chain rule)

$ \frac{\partial }{\partial w_i}\Delta = 2(y-Y)\cdot x_i$

In [32]:
gradients = np.zeros_like(weights) 


In [33]:
for i in range(weights.shape[0]):
    gradients[i] = 2 * np.dot(errors, trainingIn[:, i]) 

In [34]:
print("Gradients: ", gradients)



Gradients:  [  -197.64801479 156554.16797475 -15105.66179146]


## 3. Training: Use Gradient descent to change weights to minimize the error

Repeat the following process many time:
- Select an input pair (index)
- Calculate the gradient of the error 
- Change weights accoding to 

$ w_\mathrm{new} = w_\mathrm{old} - learingRate\cdot gradient$

In [35]:
iterations = 200000

learning_rate = 0.00001

for i in range(iterations):
    
     predictions = np.dot(trainingIn, weights)  

     loss = mse_loss(trainingOut, predictions)

     gradient = (2 / len(trainingIn)) * np.dot(trainingIn.T, (predictions - trainingOut))

     gradient = gradient.flatten()

     weights -= learning_rate * gradient


print(f"Trained Weights: {weights}")


Trained Weights: [-0.00929998  1.00002877  0.99998582]


## 4. Application to test data set (new data)

In [36]:
#lets test our model

test_predictions = np.dot(testingIn, weights)

test_loss = np.mean((testingOut - test_predictions) ** 2)

correct_predictions = np.sum(np.abs(test_predictions - testingOut))

accuracy = (correct_predictions / len(testingOut)) * 100

print(f"Trained Weights: {weights}")
print(f"Test Loss (MSE): {test_loss:.6f}")
print(f"Accuracy: {accuracy:.2f}%")



Trained Weights: [-0.00929998  1.00002877  0.99998582]
Test Loss (MSE): 0.000087
Accuracy: 0.93%
